In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve 
from sklearn.metrics import f1_score
from sklearn.metrics import auc

In [ ]:
#import file
df = pd.read_csv(os.path.join(dirname, filename))

In [ ]:
df.head()

In [ ]:
#Removing the unncessary Id column
df.drop(columns = ['id'], inplace = True)

In [ ]:
df.head()

In [ ]:
#Checking Null values
df.apply(lambda x: sum(x.isnull()),axis=0)

In [ ]:
#201 out of 5111 values are null. Dropping those null entries
df.dropna(inplace=True)

In [ ]:
#Our data containes No null values now
df.apply(lambda x: sum(x.isnull()),axis=0)

# **Exploratory Data Analysis**

In [ ]:
sns.countplot(x="stroke", data=df, palette="bwr")
plt.show()

In [ ]:
sns.countplot(x="hypertension", data=df, palette="rocket")
plt.show()

In [ ]:
sns.countplot(x="gender", data=df, palette="deep")
plt.show()

# **Preprocessing**

In [ ]:
df['gender'].value_counts()

In [ ]:
#Dropping 'Other' row values, since it has occured only once
Others = df[(df['gender'] == 'Other')].index
df.drop(Others , inplace=True)

In [ ]:
#Converting Gender into Category 0-Female, 1-Male
df["gender"] = df["gender"].astype('category')
df["gender"] = df["gender"].cat.codes

In [ ]:
#Evermarried 1-Yes, 0-No
df["ever_married"] = df["ever_married"].astype('category')
df["ever_married"] = df["ever_married"].cat.codes

In [ ]:
df['work_type'].value_counts()

In [ ]:
#Cannot form a hierarchy or preference over worktype. Hence, converted to dummies
df = pd.get_dummies(df, prefix=['w_type'], columns=['work_type'])

In [ ]:
#Anyone who used to smoke or smokes has been categorized into 1 category. THIS IS 1 APPROACH
df['smoking_status'] = df['smoking_status'].map( 
                   {'formerly smoked':1 ,'smokes':1,'never smoked':0,'Unknown':0}) 

In [ ]:
df['Residence_type'].value_counts()

In [ ]:
df = pd.get_dummies(df, prefix=['residency_'], columns=['Residence_type'])

In [ ]:
df.head()

# **Baseline Model**

In [ ]:
#Starting with Logistic Regression
#f(x) = 1/(1+e^(-x))
from sklearn.linear_model import LogisticRegression

In [ ]:
#Method for XY Split
def XYsplit(df, label_col):
    y = df[label_col].copy()
    X = df.drop(label_col,axis=1)
    return X,y

In [ ]:
X,y = XYsplit(df,'stroke')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state=0)

In [ ]:
LogReg = LogisticRegression(max_iter=1000)
LogReg.fit(X_train,y_train) 

In [ ]:
predictions = LogReg.predict(X_test)
#accuracy
accuracy = accuracy_score(y_test, predictions)*100

#precision
precision = precision_score(y_test, predictions,pos_label=1,labels=[0,1])*100
    
    #Recall
recall = recall_score(y_test, predictions,pos_label=1,labels=[0,1])*100
    
    #get FPR (specificity) and TPR (sensitivity)
fpr , tpr, _ = roc_curve(y_test, predictions)
    
    #AUC
auc_val = auc(fpr, tpr)
    
    #F-Score
f_score = f1_score(y_test, predictions)

In [ ]:
        print("Accuracy: \n", accuracy)
        print("Precision of event Happening: \n", precision)
        print("Recall of event Happening: \n", recall)
        print("AUC: \n",auc_val)
        print("F-Score:\n", f_score)
        plt.title('ROC Curve')
        plt.plot(fpr, tpr, label='AUC = {:.2f}'.format(auc_val))
        plt.plot([0,1],[0,1],'r--')
        plt.xlim([-0.1,1.1])
        plt.ylim([-0.1,1.1])
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        plt.legend(loc='lower right')
        plt.show()

In [ ]:
print("Confusion Matrix: \n", confusion_matrix(y_test, predictions,labels=[0,1]))